# Functional Spell Correction Demo (Norvig Style)

This notebook demonstrates a functional approach to spell correction using:
- N-gram Language Models with Laplace smoothing
- Noisy Channel Model
- Error Confusion Matrices
- Damerau-Levenshtein Edit Distance

We'll build up from basic components to a complete spell checker.

## 1. Import Dependencies

In [ ]:
import ast
import math
from collections import Counter

## 2. Corpus Loading

First, let's load and process a corpus of text.

In [ ]:
def load_corpus(filename='corpus.data'):
    """Load and tokenize corpus from file."""
    print(f"Loading Corpus from {filename}")
    with open(filename, 'r') as f:
        corpus = f.read()
    print("Processing Corpus")
    return corpus.split(' ')

# Demo: Load a small sample
words = load_corpus()
print(f"Total words: {len(words)}")
print(f"First 10 words: {words[:10]}")

## 3. N-gram Creation

Create unigram and bigram models using Counter.

In [ ]:
def create_unigram(words):
    """Create Unigram Model from word list."""
    print("Creating Unigram Model")
    return Counter(words)

def create_bigram(words):
    """Create Bigram Model from word list."""
    print("Creating Bigram Model")
    return Counter(' '.join(words[i:i+2]) for i in range(len(words)-1))

# Demo: Create models
unigram = create_unigram(words)
bigram = create_bigram(words)

print(f"Unique words: {len(unigram)}")
print(f"Most common words: {unigram.most_common(5)}")
print(f"\nMost common bigrams: {bigram.most_common(5)}")

## 4. Complete N-gram Model Creation

Build all n-gram models in one go.

In [ ]:
def create_ngrams(gram_types=['uni', 'bi'], words=None, corpus_file='corpus.data'):
    """Create specified n-gram models."""
    if words is None:
        words = load_corpus(corpus_file)
    
    models = {'words': words}
    
    creators = {
        'uni': create_unigram,
        'bi': create_bigram
    }
    
    for gram_type in gram_types:
        if gram_type in creators:
            models[gram_type] = creators[gram_type](words)
    
    return models

# Demo: Create models
models = create_ngrams(['uni', 'bi'], words=words)
print("Models created:", list(models.keys()))

## 5. Probability Calculation

Calculate log probabilities with Laplace smoothing.

In [ ]:
def probability(word, context="", gram_type='uni', models=None):
    """Calculate Maximum Likelihood Probability with Laplace smoothing."""
    if models is None:
        raise ValueError("models dictionary required")
    
    words = models['words']
    vocab_size = len(models.get('uni', Counter()))
    
    if gram_type == 'uni':
        return math.log((models['uni'][word] + 1) / (len(words) + vocab_size))
    elif gram_type == 'bi':
        bigram_count = models['bi'][context] + 1
        unigram_count = models['uni'][word] + vocab_size
        return math.log(bigram_count / unigram_count)

# Demo: Calculate probabilities
word_prob = probability('the', gram_type='uni', models=models)
bigram_prob = probability('the', 'the actress', gram_type='bi', models=models)

print(f"P(the) = {math.exp(word_prob):.6f} (log: {word_prob:.4f})")
print(f"P(actress|the) = {math.exp(bigram_prob):.6f} (log: {bigram_prob:.4f})")

## 6. Sentence Probability

Calculate cumulative probability for a full sentence.

In [ ]:
def sentence_probability(sentence, models, gram_type='uni', form='antilog'):
    """Calculate cumulative n-gram probability for a sentence."""
    words = sentence.lower().split()
    n = len(words)
    
    if n < 1:
        return 0 if form == 'log' else 1
    
    log_prob = 0
    
    if gram_type == 'uni':
        log_prob = sum(probability(w, "", 'uni', models) for w in words)
    elif gram_type == 'bi':
        for i in range(n - 1):
            context = f"{words[i]} {words[i+1]}"
            log_prob += probability(words[i], context, 'bi', models)
    
    return log_prob if form == 'log' else math.exp(log_prob)

# Demo: Compare sentences
sent1 = "the actress is brilliant"
sent2 = "brilliant actress the is"

prob1 = sentence_probability(sent1, models, 'bi', 'log')
prob2 = sentence_probability(sent2, models, 'bi', 'log')

print(f"'{sent1}': {prob1:.4f}")
print(f"'{sent2}': {prob2:.4f}")
print(f"\nFirst sentence is {'more' if prob1 > prob2 else 'less'} likely!")

## 7. Load Confusion Matrices

These matrices model common typing errors.

In [ ]:
def load_confusion_matrices():
    """Load all confusion matrices from data files."""
    print("Loading confusion matrices")
    matrices = {}
    
    for matrix_type in ['add', 'sub', 'rev', 'del']:
        filename = f"{matrix_type}confusion.data"
        with open(filename, 'r') as f:
            matrices[matrix_type] = ast.literal_eval(f.read())
    
    return matrices

# Demo: Load matrices
matrices = load_confusion_matrices()
print("Matrix types:", list(matrices.keys()))
print(f"Addition matrix size: {len(matrices['add'])}")
print(f"Example - 'th' added: {matrices['add'].get('th', 0)} times")

## 8. Damerau-Levenshtein Edit Distance

Measures the minimum edits needed to transform one string to another.

In [ ]:
def damerau_levenshtein_distance(s1, s2):
    """Calculate Damerau-Levenshtein Edit Distance between two strings."""
    s1, s2 = '#' + s1, '#' + s2
    m, n = len(s1), len(s2)
    
    D = [[0] * n for _ in range(m)]
    for i in range(m):
        D[i][0] = i
    for j in range(n):
        D[0][j] = j
    
    for i in range(1, m):
        for j in range(1, n):
            costs = [
                D[i-1][j] + 1,
                D[i][j-1] + 1,
                D[i-1][j-1] + (2 if s1[i] != s2[j] else 0)
            ]
            
            if i > 1 and j > 1 and s1[i] == s2[j-1] and s1[i-1] == s2[j]:
                costs.append(D[i-2][j-2] + 1)
            
            D[i][j] = min(costs)
    
    return D[m-1][n-1]

# Demo: Test edit distances
print(f"'actress' → 'acress': {damerau_levenshtein_distance('actress', 'acress')}")
print(f"'brilliant' → 'briliant': {damerau_levenshtein_distance('brilliant', 'briliant')}")
print(f"'hello' → 'helo': {damerau_levenshtein_distance('hello', 'helo')}")
print(f"'actor' → 'acotr': {damerau_levenshtein_distance('actor', 'acotr')}")

## 9. Generate Candidate Corrections

Find words within edit distance 1 of the misspelled word.

In [ ]:
def generate_candidates(word, words, max_distance=1):
    """Generate candidate corrections within edit distance threshold."""
    candidates = {}
    for w in words:
        distance = damerau_levenshtein_distance(word, w)
        if distance <= max_distance:
            candidates[w] = distance
    return sorted(candidates, key=candidates.get)

# Demo: Create vocabulary and find candidates
vocab = sorted(set(words))[3246:]  # Skip common words
print(f"Vocabulary size: {len(vocab)}")

misspelled = "acress"
candidates = generate_candidates(misspelled, vocab)
print(f"\nCandidates for '{misspelled}': {candidates[:10]}")

## 10. Detect Edit Type

Identify what kind of error was made (insertion, deletion, substitution, or transposition).

In [ ]:
def detect_edit_type(candidate, word):
    """Detect the type of edit between strings."""
    
    def check_edits(cand, wrd, reverse=False):
        for i in range(min(len(wrd), len(cand)) - 1):
            if cand[0:i+1] != wrd[0:i+1]:
                if cand[i:] == wrd[i-1:]:
                    correct = cand[i-1]
                    x = cand[i-2] if i > 1 else ''
                    w = x + correct
                    return ("Deletion", correct, '', x, w)
                elif cand[i:] == wrd[i+1:]:
                    error = wrd[i]
                    if i == 0:
                        w, x = '#', '#' + error
                    else:
                        w, x = wrd[i-1], wrd[i-1] + error
                    return ("Insertion", '', error, x, w)
                if i + 1 < len(cand) and i + 1 < len(wrd) and cand[i+1:] == wrd[i+1:]:
                    correct, error = cand[i], wrd[i]
                    return ("Substitution", correct, error, error, correct)
                if (i + 1 < len(wrd) and i + 2 <= len(cand) and 
                    cand[i] == wrd[i+1] and i + 2 <= len(wrd) and cand[i+2:] == wrd[i+2:]):
                    correct = cand[i] + cand[i+1]
                    error = wrd[i] + wrd[i+1]
                    return ("Reversal", correct, error, error, correct)
        return None
    
    if word == candidate:
        return ("None", '', '', '', '')
    
    result = check_edits(candidate, word)
    if result:
        return result
    
    result = check_edits(candidate[::-1], word[::-1], reverse=True)
    if result:
        return result
    
    return ("None", '', '', '', '')

# Demo: Detect edits
print("actress → acress:", detect_edit_type('actress', 'acress')[0])
print("brilliant → briliant:", detect_edit_type('brilliant', 'briliant')[0])
print("actor → acotr:", detect_edit_type('actor', 'acotr')[0])

## 11. Channel Model Probability

Calculate the probability that a user would make this specific error.

In [ ]:
def channel_model_probability(x, y, edit_type, matrices, corpus):
    """Calculate channel model probability for errors using confusion matrices."""
    if edit_type == 'add':
        if x == '#':
            count = corpus.count(' ' + y)
        else:
            count = corpus.count(x)
        return matrices['add'].get(x + y, 0) / max(count, 1)
    elif edit_type == 'sub':
        key = (x + y)[0:2]
        count = corpus.count(y)
        return matrices['sub'].get(key, 0) / max(count, 1)
    elif edit_type == 'rev':
        count = corpus.count(x + y)
        return matrices['rev'].get(x + y, 0) / max(count, 1)
    elif edit_type == 'del':
        count = corpus.count(x + y)
        return matrices['del'].get(x + y, 0) / max(count, 1)
    return 0.0

# Demo: Calculate channel probabilities
corpus_str = ' '.join(words)
prob_del = channel_model_probability('t', 'tr', 'del', matrices, corpus_str)
print(f"P(deleting 't' from 'tr'): {prob_del:.8f}")

## 12. Word Correction with Noisy Channel Model

Combine channel model and language model to pick the best correction.

In [ ]:
def correct_word(word, prev_word, next_word, context):
    """Correct a single word using noisy channel model."""
    candidates = generate_candidates(word, context['words'])
    
    if word in candidates:
        return word
    
    scores = {}
    
    for candidate in candidates:
        edit_info = detect_edit_type(candidate, word)
        
        if edit_info[0] == "None":
            continue
        
        edit_type_map = {
            "Insertion": ('add', edit_info[3][0] if len(edit_info[3]) > 0 else '', 
                         edit_info[3][1] if len(edit_info[3]) > 1 else ''),
            "Deletion": ('del', edit_info[4][0] if len(edit_info[4]) > 0 else '', 
                        edit_info[4][1] if len(edit_info[4]) > 1 else ''),
            "Reversal": ('rev', edit_info[4][0] if len(edit_info[4]) > 0 else '', 
                        edit_info[4][1] if len(edit_info[4]) > 1 else ''),
            "Substitution": ('sub', edit_info[3], edit_info[4])
        }
        
        if edit_info[0] not in edit_type_map:
            continue
        
        edit_type, x, y = edit_type_map[edit_info[0]]
        channel_prob = channel_model_probability(x, y, edit_type, 
                                                 context['matrices'], context['corpus'])
        
        if next_word:
            phrase = f"{prev_word} {candidate} {next_word}" if prev_word else f"{candidate} {next_word}"
        else:
            phrase = f"{prev_word} {candidate}" if prev_word else candidate
        
        try:
            lm_prob = math.exp(sentence_probability(phrase, context['models'], 'bi', 'log'))
        except:
            lm_prob = 1e-10
        
        scores[candidate] = channel_prob * lm_prob * 1e9
    
    if scores:
        return max(scores, key=scores.get)
    return ''

# Demo: Prepare context
context = {
    'models': models,
    'words': vocab,
    'matrices': matrices,
    'corpus': corpus_str
}

# Test word correction
corrected = correct_word('acress', 'a', '', context)
print(f"'acress' → '{corrected}'")

## 13. Sentence Correction

Now let's correct an entire sentence!

In [ ]:
def correct_sentence(sentence, context):
    """Correct spelling errors in a sentence."""
    words = sentence.lower().split()
    corrected = []
    
    for i, word in enumerate(words):
        prev_word = words[i-1] if i > 0 else ''
        next_word = words[i+1] if i < len(words) - 1 else ''
        
        corrected_word = correct_word(word, prev_word, next_word, context)
        corrected.append(corrected_word if corrected_word else word)
    
    return ' '.join(corrected)

# Demo: Correct the example sentence
test_sentence = "she is a briliant acress"
corrected_sentence = correct_sentence(test_sentence, context)

print(f"Original:  '{test_sentence}'")
print(f"Corrected: '{corrected_sentence}'")

## 14. Additional Test Cases

Let's try a few more examples to see how well it works!

In [ ]:
test_cases = [
    "she is a briliant acress",
    "the acotr was gerat",
    "hold your horeses",
]

print("Spell Correction Results:")
print("=" * 60)
for test in test_cases:
    corrected = correct_sentence(test, context)
    print(f"Input:  {test}")
    print(f"Output: {corrected}")
    print("-" * 60)

## Summary

This functional approach to spell correction combines:

1. **N-gram Language Models**: Predict likely word sequences
2. **Damerau-Levenshtein Distance**: Find words within 1 edit
3. **Confusion Matrices**: Model common typing errors
4. **Noisy Channel Model**: Combine error probability × language model probability

The result is a powerful spell checker that considers both the likelihood of the typo and the linguistic context!